# Experiment 006: Grid-Based Translation Approach (egortrushin)

Implementing the grid-based translation approach that builds configurations from scratch.
This is fundamentally different from SA on existing configurations.

In [1]:
import math
import random
import numpy as np
import pandas as pd
from decimal import Decimal, getcontext
from shapely import affinity
from shapely.geometry import Polygon
from shapely.ops import unary_union
import time

getcontext().prec = 30
scale_factor = Decimal('1e15')

print("Libraries loaded")

Libraries loaded


In [2]:
class ChristmasTree:
    def __init__(self, center_x='0', center_y='0', angle='0'):
        self.center_x = Decimal(str(center_x))
        self.center_y = Decimal(str(center_y))
        self.angle = Decimal(str(angle))
        trunk_w = Decimal('0.15')
        trunk_h = Decimal('0.2')
        base_w = Decimal('0.7')
        mid_w = Decimal('0.4')
        top_w = Decimal('0.25')
        tip_y = Decimal('0.8')
        tier_1_y = Decimal('0.5')
        tier_2_y = Decimal('0.25')
        base_y = Decimal('0.0')
        trunk_bottom_y = -trunk_h
        initial_polygon = Polygon([
            (Decimal('0.0') * scale_factor, tip_y * scale_factor),
            (top_w / Decimal('2') * scale_factor, tier_1_y * scale_factor),
            (top_w / Decimal('4') * scale_factor, tier_1_y * scale_factor),
            (mid_w / Decimal('2') * scale_factor, tier_2_y * scale_factor),
            (mid_w / Decimal('4') * scale_factor, tier_2_y * scale_factor),
            (base_w / Decimal('2') * scale_factor, base_y * scale_factor),
            (trunk_w / Decimal('2') * scale_factor, base_y * scale_factor),
            (trunk_w / Decimal('2') * scale_factor, trunk_bottom_y * scale_factor),
            (-(trunk_w / Decimal('2')) * scale_factor, trunk_bottom_y * scale_factor),
            (-(trunk_w / Decimal('2')) * scale_factor, base_y * scale_factor),
            (-(base_w / Decimal('2')) * scale_factor, base_y * scale_factor),
            (-(mid_w / Decimal('4')) * scale_factor, tier_2_y * scale_factor),
            (-(mid_w / Decimal('2')) * scale_factor, tier_2_y * scale_factor),
            (-(top_w / Decimal('4')) * scale_factor, tier_1_y * scale_factor),
            (-(top_w / Decimal('2')) * scale_factor, tier_1_y * scale_factor),
        ])
        rotated = affinity.rotate(initial_polygon, float(self.angle), origin=(0, 0))
        self.polygon = affinity.translate(rotated, xoff=float(self.center_x * scale_factor), yoff=float(self.center_y * scale_factor))

    def clone(self):
        return ChristmasTree(str(self.center_x), str(self.center_y), str(self.angle))

print("ChristmasTree class defined")

ChristmasTree class defined


In [3]:
def get_side_length(trees):
    all_polygons = [t.polygon for t in trees]
    bounds = unary_union(all_polygons).bounds
    return max(bounds[2] - bounds[0], bounds[3] - bounds[1]) / float(scale_factor)

def get_score(trees, n):
    if not trees:
        return 0.0
    side = get_side_length(trees)
    return side ** 2 / n

def has_collision(trees):
    """Check for collisions using Shapely (ground truth)."""
    if len(trees) <= 1:
        return False
    for i, tree1 in enumerate(trees):
        for j, tree2 in enumerate(trees):
            if i < j:
                if tree1.polygon.intersects(tree2.polygon) and not tree1.polygon.touches(tree2.polygon):
                    return True
    return False

def load_configuration_from_df(n, df):
    group_data = df[df["id"].str.startswith(f"{n:03d}_")]
    trees = []
    for _, row in group_data.iterrows():
        x = str(row["x"])[1:] if str(row["x"]).startswith('s') else str(row["x"])
        y = str(row["y"])[1:] if str(row["y"]).startswith('s') else str(row["y"])
        deg = str(row["deg"])[1:] if str(row["deg"]).startswith('s') else str(row["deg"])
        if x and y and deg:
            trees.append(ChristmasTree(x, y, deg))
    return trees

print("Helper functions defined")

Helper functions defined


In [4]:
# Verify the repaired baseline
print("Verifying repaired baseline...")
df = pd.read_csv('/home/submission/submission.csv')

total_score = 0
overlap_count = 0
for n in range(1, 201):
    trees = load_configuration_from_df(n, df)
    if trees:
        score = get_score(trees, n)
        total_score += score
        if has_collision(trees):
            overlap_count += 1
            if overlap_count <= 5:
                print(f"  N={n}: OVERLAP!")

print(f"\nTotal score: {total_score:.6f}")
print(f"Overlaps: {overlap_count}")
print(f"Gap to target: {total_score - 68.922808:.6f} points")

Verifying repaired baseline...



Total score: 70.682741
Overlaps: 0
Gap to target: 1.759933 points


In [5]:
# Grid-based translation approach from egortrushin kernel
# Creates trees in a grid pattern and optimizes the translation parameters

def create_grid_trees(n, dx, dy, angle1=0, angle2=180):
    """
    Create n trees in a grid pattern with alternating angles.
    dx, dy are the translation between adjacent trees.
    """
    trees = []
    
    # Find grid dimensions
    cols = int(math.ceil(math.sqrt(n * 1.5)))  # More columns than rows
    rows = int(math.ceil(n / cols))
    
    count = 0
    for row in range(rows):
        for col in range(cols):
            if count >= n:
                break
            
            # Alternating angles for interlocking
            angle = angle1 if (row + col) % 2 == 0 else angle2
            
            # Position with offset for odd rows
            x = col * dx
            if row % 2 == 1:
                x += dx / 2  # Offset for interlocking
            y = row * dy
            
            trees.append(ChristmasTree(str(x), str(y), str(angle)))
            count += 1
        if count >= n:
            break
    
    return trees

def optimize_grid_sa(n, n_steps=5000, T_max=1.0, T_min=0.001, seed=42):
    """
    Optimize grid parameters using simulated annealing.
    """
    random.seed(seed)
    
    # Initial parameters (based on tree dimensions)
    # Tree width is ~0.7, height is ~1.0
    best_dx = 0.7
    best_dy = 0.9
    best_angle1 = 0
    best_angle2 = 180
    
    trees = create_grid_trees(n, best_dx, best_dy, best_angle1, best_angle2)
    if has_collision(trees):
        # Increase spacing until no collision
        for scale in [1.1, 1.2, 1.3, 1.4, 1.5, 2.0]:
            trees = create_grid_trees(n, best_dx * scale, best_dy * scale, best_angle1, best_angle2)
            if not has_collision(trees):
                best_dx *= scale
                best_dy *= scale
                break
    
    best_side = get_side_length(trees)
    best_score = get_score(trees, n)
    
    # SA parameters
    alpha = (T_min / T_max) ** (1.0 / n_steps)
    T = T_max
    
    current_dx, current_dy = best_dx, best_dy
    current_angle1, current_angle2 = best_angle1, best_angle2
    current_side = best_side
    
    for step in range(n_steps):
        # Perturb parameters
        scale = T / T_max
        new_dx = current_dx + random.uniform(-0.05 * scale, 0.05 * scale)
        new_dy = current_dy + random.uniform(-0.05 * scale, 0.05 * scale)
        new_angle1 = (current_angle1 + random.uniform(-10 * scale, 10 * scale)) % 360
        new_angle2 = (current_angle2 + random.uniform(-10 * scale, 10 * scale)) % 360
        
        # Ensure positive spacing
        new_dx = max(0.3, new_dx)
        new_dy = max(0.3, new_dy)
        
        # Create new configuration
        trees = create_grid_trees(n, new_dx, new_dy, new_angle1, new_angle2)
        
        if not has_collision(trees):
            new_side = get_side_length(trees)
            delta = new_side - current_side
            
            if delta < 0 or random.random() < math.exp(-delta / T):
                current_dx, current_dy = new_dx, new_dy
                current_angle1, current_angle2 = new_angle1, new_angle2
                current_side = new_side
                
                if new_side < best_side:
                    best_dx, best_dy = new_dx, new_dy
                    best_angle1, best_angle2 = new_angle1, new_angle2
                    best_side = new_side
        
        T *= alpha
    
    # Return best configuration
    trees = create_grid_trees(n, best_dx, best_dy, best_angle1, best_angle2)
    return trees, best_side

print("Grid optimization functions defined")

Grid optimization functions defined


In [6]:
# Test grid approach on a few N values
print("Testing grid approach...")

test_ns = [10, 20, 50, 100]
for n in test_ns:
    baseline_trees = load_configuration_from_df(n, df)
    baseline_score = get_score(baseline_trees, n)
    
    start = time.time()
    grid_trees, grid_side = optimize_grid_sa(n, n_steps=3000, seed=42)
    elapsed = time.time() - start
    
    grid_score = get_score(grid_trees, n)
    has_overlap = has_collision(grid_trees)
    
    print(f"N={n}: baseline={baseline_score:.6f}, grid={grid_score:.6f}, overlap={has_overlap}, time={elapsed:.1f}s")

Testing grid approach...


N=10: baseline=0.376630, grid=1.088212, overlap=False, time=4.3s


N=20: baseline=0.376057, grid=0.989167, overlap=False, time=10.5s


N=50: baseline=0.360753, grid=0.843694, overlap=False, time=38.0s


N=100: baseline=0.345531, grid=0.837886, overlap=False, time=115.7s


In [ ]:
# Run grid optimization on all N values and compare with baseline
print("\nRunning grid optimization on all N values...")

best_configs = {}
best_scores = {}
improvements = 0

start_time = time.time()

for n in range(1, 201):
    baseline_trees = load_configuration_from_df(n, df)
    baseline_score = get_score(baseline_trees, n)
    
    # Adjust iterations based on N
    if n <= 20:
        n_steps = 5000
    elif n <= 50:
        n_steps = 3000
    else:
        n_steps = 2000
    
    grid_trees, grid_side = optimize_grid_sa(n, n_steps=n_steps, seed=42+n)
    grid_score = get_score(grid_trees, n)
    
    # Use grid if better and no overlap
    if grid_score < baseline_score and not has_collision(grid_trees):
        best_configs[n] = grid_trees
        best_scores[n] = grid_score
        improvements += 1
        if n % 20 == 0:
            print(f"N={n}: IMPROVED {baseline_score:.6f} -> {grid_score:.6f}")
    else:
        best_configs[n] = baseline_trees
        best_scores[n] = baseline_score
        if n % 20 == 0:
            print(f"N={n}: kept baseline {baseline_score:.6f} (grid={grid_score:.6f})")

elapsed = time.time() - start_time
print(f"\nTotal time: {elapsed:.1f}s")
print(f"Improvements: {improvements}")

new_total = sum(best_scores.values())
print(f"\nNew total score: {new_total:.6f}")
print(f"Improvement: {total_score - new_total:.6f} points")
print(f"Gap to target: {new_total - 68.922808:.6f} points")